Goal : Predict rain sum (mm)

# Library

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

# Loading Data

In [9]:
train_data = pd.read_csv('train.csv')
train_h_data = pd.read_csv('train_hourly.csv')
test_data = pd.read_csv('test.csv')
test_h_data = pd.read_csv('test_hourly.csv')

In [5]:
train_data.head()

,time,temperature_2m_max (°C),temperature_2m_min (°C),apparent_temperature_max (°C),apparent_temperature_min (°C),sunrise (iso8601),sunset (iso8601),shortwave_radiation_sum (MJ/m²),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),winddirection_10m_dominant (°),et0_fao_evapotranspiration (mm),elevation,city
0,2018-01-01,29.9,26.0,36.3,31.6,2018-01-01T05:15,2018-01-01T17:49,17.59,7.7,0.0,6.9,20.2,277.0,3.61,0,su
1,2018-01-02,30.6,25.7,37.7,31.2,2018-01-02T05:15,2018-01-02T17:50,19.87,9.4,0.0,7.1,18.0,168.0,4.11,0,su
2,2018-01-03,31.8,25.9,40.3,31.7,2018-01-03T05:16,2018-01-03T17:50,20.44,7.8,0.0,8.1,21.2,125.0,4.23,0,su
3,2018-01-04,30.8,26.0,39.1,31.6,2018-01-04T05:16,2018-01-04T17:50,20.44,29.7,0.0,7.2,21.6,190.0,4.22,0,su
4,2018-01-05,30.9,25.2,37.6,29.9,2018-01-05T05:17,2018-01-05T17:51,20.10,22.5,0.0,6.9,21.2,241.0,4.16,0,su


In [30]:
print("Train Data Total: " + str(len(train_data)))
print("Train Hourly Data Total: " + str(len(train_h_data)))

Train Data Total: 13258
Train Hourly Data Total: 318192


In [8]:
train_data.columns

Index(['time', 'temperature_2m_max (°C)', 'temperature_2m_min (°C)',
       'apparent_temperature_max (°C)', 'apparent_temperature_min (°C)',
       'sunrise (iso8601)', 'sunset (iso8601)',
       'shortwave_radiation_sum (MJ/m²)', 'rain_sum (mm)', 'snowfall_sum (cm)',
       'windspeed_10m_max (km/h)', 'windgusts_10m_max (km/h)',
       'winddirection_10m_dominant (°)', 'et0_fao_evapotranspiration (mm)',
       'elevation', 'city'],
      dtype='object')

In [19]:
train_h_data.columns

Index(['time', 'temperature_2m (°C)', 'relativehumidity_2m (%)',
       'dewpoint_2m (°C)', 'apparent_temperature (°C)', 'pressure_msl (hPa)',
       'surface_pressure (hPa)', 'snowfall (cm)', 'cloudcover (%)',
       'cloudcover_low (%)', 'cloudcover_mid (%)', 'cloudcover_high (%)',
       'shortwave_radiation (W/m²)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)', 'direct_normal_irradiance (W/m²)',
       'windspeed_10m (km/h)', 'windspeed_100m (km/h)',
       'winddirection_10m (°)', 'winddirection_100m (°)',
       'windgusts_10m (km/h)', 'et0_fao_evapotranspiration (mm)',
       'vapor_pressure_deficit (kPa)', 'soil_temperature_0_to_7cm (°C)',
       'soil_temperature_7_to_28cm (°C)', 'soil_temperature_28_to_100cm (°C)',
       'soil_temperature_100_to_255cm (°C)', 'soil_moisture_0_to_7cm (m³/m³)',
       'soil_moisture_7_to_28cm (m³/m³)', 'soil_moisture_28_to_100cm (m³/m³)',
       'soil_moisture_100_to_255cm (m³/m³)', 'city'],
      dtype='object')

# Data Cleansing

In [15]:
train_data.isnull().sum()

time                                 0
temperature_2m_max (°C)             50
temperature_2m_min (°C)             50
apparent_temperature_max (°C)       50
apparent_temperature_min (°C)       50
sunrise (iso8601)                    0
sunset (iso8601)                     0
shortwave_radiation_sum (MJ/m²)     60
rain_sum (mm)                       60
snowfall_sum (cm)                   60
windspeed_10m_max (km/h)            50
windgusts_10m_max (km/h)            50
winddirection_10m_dominant (°)     466
et0_fao_evapotranspiration (mm)     60
elevation                            0
city                                 0
dtype: int64

In [25]:
train_h_data.isnull().sum()

time                                    0
temperature_2m (°C)                   170
relativehumidity_2m (%)               170
dewpoint_2m (°C)                      170
apparent_temperature (°C)             170
pressure_msl (hPa)                    170
surface_pressure (hPa)                170
snowfall (cm)                         170
cloudcover (%)                        170
cloudcover_low (%)                    170
cloudcover_mid (%)                    170
cloudcover_high (%)                   170
shortwave_radiation (W/m²)            170
direct_radiation (W/m²)               170
diffuse_radiation (W/m²)              170
direct_normal_irradiance (W/m²)       160
windspeed_10m (km/h)                  170
windspeed_100m (km/h)                 170
winddirection_10m (°)                 602
winddirection_100m (°)                347
windgusts_10m (km/h)                  170
et0_fao_evapotranspiration (mm)       170
vapor_pressure_deficit (kPa)          170
soil_temperature_0_to_7cm (°C)    